# 파이썬으로 엑셀다루기 3 (with Pandas)
_본 자료는 안수찬 강사님의 파이썬을 활용한 업무자동화 Camp (fast campus)의 강의자료를 기반으로 만들어졌습니다._  
만든이 : 김보섭  

_**전국 대학교의 리스트, 전국 대학교에 포함된 학과의 리스트, 각각의 대학에 대한 지역에 관한 실제 데이터로 아래의 두 예제를 직접 풀어봅니다.**_  

_(boolean mask를 이용해서 filtering 하는 방법, groupby를 활용하는 방법을 이용해봅니다.)_  

  
* Example 1. 대학교별로 나눠보고 싶다. eg) '경북대학교.xlsx' * 1500  
* Example 2. 지역별로 나누고, 폴더로 구분하기 '경북/경북대학교.xlsx'

### Example 1. 대학교라는 폴더안에 각각의 학교 별 excel 파일을 저장하기

In [1]:
import os, sys, re
import shutil
import pandas as pd

In [2]:
df = pd.read_excel('./데이터/univs_2014.xlsx')

In [3]:
df.head()

,연도,학제,시도,학교명,본분교,학교상태,설립,우편번호,주소,전화번호,팩스번호,홈페이지,학과 계열,Unnamed: 13,Unnamed: 14,학과상태,학과명
0,2014,전문대학(3년제),충남,충남도립청양대학,본교,기존,공립,345702,"충청남도 청양군 학사길 55 (청양읍, 청양대학)",041-635-6600,041-635-6633,www.cyc.ac.kr,사회계열,경영ㆍ경제,경영ㆍ경제,기존,토지행정과
1,2014,전문대학(3년제),충남,충남도립청양대학,본교,기존,공립,345702,"충청남도 청양군 학사길 55 (청양읍, 청양대학)",041-635-6600,041-635-6633,www.cyc.ac.kr,사회계열,사회과학,행정,기존,경찰행정과
2,2014,전문대학(3년제),충남,충남도립청양대학,본교,기존,공립,345702,"충청남도 청양군 학사길 55 (청양읍, 청양대학)",041-635-6600,041-635-6633,www.cyc.ac.kr,사회계열,사회과학,행정,기존,자치행정과
3,2014,전문대학(3년제),충남,충남도립청양대학,본교,기존,공립,345702,"충청남도 청양군 학사길 55 (청양읍, 청양대학)",041-635-6600,041-635-6633,www.cyc.ac.kr,사회계열,사회과학,행정,기존,자치행정학과
4,2014,전문대학(3년제),충남,충남도립청양대학,본교,기존,공립,345702,"충청남도 청양군 학사길 55 (청양읍, 청양대학)",041-635-6600,041-635-6633,www.cyc.ac.kr,공학계열,전기ㆍ전자,전자,기존,전기전자과


In [4]:
df.shape

(20017, 17)

In [5]:
# unique_univ 대학교 수?
unique_univs = list(set(df['학교명']))
len(unique_univs)

1592

In [6]:
# boolean Mask를 이용하여 필터링 하는 법
is_univ = df['학교명'] == unique_univs[1]
print(unique_univs[1])
df[is_univ]['학과명']

홍익대학교 세종캠퍼스 산업대학원


19222    광고홍보커뮤니케이션전공
19223          건축설계전공
19224     커뮤니케이션디자인전공
19225        게임프로듀싱전공
19226            색채전공
Name: 학과명, dtype: object

#### 대학교/ 대학교별 엑셀

In [7]:
# 대학교/ 대학교별 엑셀
if '대학교' in os.listdir():
    shutil.rmtree('대학교')
os.makedirs('대학교')

unique_univs = list(set(df['학교명']))
for univ_name in unique_univs:
    is_univ = df['학교명'] == univ_name
    univ_df = df[is_univ]
    univ_df.to_excel('./대학교/' + univ_name + '.xlsx')

In [8]:
# 더 효율적으로 해보자
# list(set(df['학교명'])) 부분을 refactoring 하자!
# boolean mask를 이용한 필터링을 하지말고 groupby를 활용하자.
import os, sys
if '대학교' in os.listdir():
    shutil.rmtree('대학교')
os.makedirs('대학교')

for univ_name in df['학교명'].unique():
    univ_df = df.groupby('학교명').get_group(univ_name)
    univ_df.to_excel('./대학교/' + univ_name + '.xlsx')

#### 성균관대학교 경영대학원 같은 case가 존재하므로 위의 case 들도 고려해야한다. 그래서 다음과 같은 코드를 사용
 

In [9]:
univ_names = df['학교명'].unique()
len(univ_names)

1592

In [10]:
univ_total_text = ' '.join(univ_names)

In [11]:
univ_pattern = re.compile('\w+대학교')
real_univ_names = list(set(univ_pattern.findall(univ_total_text)))

In [12]:
if '대학교' in os.listdir():
    shutil.rmtree('대학교')
os.makedirs('대학교')

for univ_name in real_univ_names:
    is_univ = df['학교명'].str.startswith(univ_name)
    univ_df = df[is_univ]
    univ_df.to_excel('./대학교/' + univ_name + '.xlsx')    

In [13]:
len(os.listdir('./대학교/'))

384

### Example 2. 대학교 폴더안에 지역 별 폴더로 나누고, 해당 폴더안에 지역별 대학교 excel sheet 저장

In [14]:
# os.path.join을 이용하자.
if '대학교' in os.listdir():
    shutil.rmtree('대학교')
os.makedirs('대학교')

city_groups = df.groupby('시도')
for city_name in df['시도'].unique():
    city_df = city_groups.get_group(city_name)
    
    if not city_name in os.listdir('대학교'):
        os.makedirs(os.path.join('대학교', city_name))
        
    univ_in_city_groups = city_df.groupby('학교명')
    
    for univ_name in city_df['학교명'].unique():
        univ_in_city_df = univ_in_city_groups.get_group(univ_name)
        univ_in_city_df.to_excel(os.path.join('대학교', city_name, univ_name + '.xlsx'))